In [30]:
data <- read.csv("../data/support2.csv")
head(data)

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,⋯,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
,<dbl>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<dbl>
1,62.84998,0,male,0,5,2029,Lung Cancer,Cancer,0,11,⋯,1.1999512,141,7.459961,NA,NA,NA,7,7,,7
2,60.33899,1,female,1,4,4,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,5.5000000,132,7.250000,NA,NA,NA,NA,1,<2 mo. follow-up,1
3,52.74698,1,female,0,17,47,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,2.0000000,134,7.459961,NA,NA,NA,1,0,<2 mo. follow-up,0
4,42.38498,1,female,0,3,133,Lung Cancer,Cancer,2,11,⋯,0.7999268,139,NA,NA,NA,NA,0,0,no(M2 and SIP pres),0
5,79.88495,0,female,0,16,2029,ARF/MOSF w/Sepsis,ARF/MOSF,1,NA,⋯,0.7999268,143,7.509766,NA,NA,NA,NA,2,no(M2 and SIP pres),2
6,93.01599,1,male,1,4,4,Coma,Coma,1,14,⋯,0.6999512,140,7.659180,NA,NA,NA,NA,1,<2 mo. follow-up,1


In [31]:
data$pafi[is.na(data$pafi)] <- 333.3
data$alb[is.na(data$alb)] <- 3.5
data$bun[is.na(data$bun)] <- 6.51
data$urine[is.na(data$urine)] <- 2502

In [32]:
data <- data[, !names(data) %in% "dnr"]
data <- data[, !names(data) %in% "scoma"]
data <- data[, !names(data) %in% "sps"]
data <- data[, !names(data) %in% "surv2m"]
data <- data[, !names(data) %in% "surv6m"]

head(data)

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,⋯,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
,<dbl>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<dbl>
1,62.84998,0,male,0,5,2029,Lung Cancer,Cancer,0,11,⋯,1.1999512,141,7.459961,NA,6.51,2502,7,7,,7
2,60.33899,1,female,1,4,4,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,5.5000000,132,7.250000,NA,6.51,2502,NA,1,<2 mo. follow-up,1
3,52.74698,1,female,0,17,47,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,2.0000000,134,7.459961,NA,6.51,2502,1,0,<2 mo. follow-up,0
4,42.38498,1,female,0,3,133,Lung Cancer,Cancer,2,11,⋯,0.7999268,139,NA,NA,6.51,2502,0,0,no(M2 and SIP pres),0
5,79.88495,0,female,0,16,2029,ARF/MOSF w/Sepsis,ARF/MOSF,1,NA,⋯,0.7999268,143,7.509766,NA,6.51,2502,NA,2,no(M2 and SIP pres),2
6,93.01599,1,male,1,4,4,Coma,Coma,1,14,⋯,0.6999512,140,7.659180,NA,6.51,2502,NA,1,<2 mo. follow-up,1


In [33]:
data[] <- lapply(data, function(x) {
  if (is.numeric(x)) {
    x[is.na(x)] <- median(x, na.rm = TRUE)
  }
  return(x)
})
head(data)

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,⋯,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
,<dbl>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,62.84998,0,male,0,5,2029,Lung Cancer,Cancer,0,11,⋯,1.1999512,141,7.459961,135,6.51,2502,7,7,,7
2,60.33899,1,female,1,4,4,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,5.5000000,132,7.250000,135,6.51,2502,0,1,<2 mo. follow-up,1
3,52.74698,1,female,0,17,47,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,2.0000000,134,7.459961,135,6.51,2502,1,0,<2 mo. follow-up,0
4,42.38498,1,female,0,3,133,Lung Cancer,Cancer,2,11,⋯,0.7999268,139,7.419922,135,6.51,2502,0,0,no(M2 and SIP pres),0
5,79.88495,0,female,0,16,2029,ARF/MOSF w/Sepsis,ARF/MOSF,1,12,⋯,0.7999268,143,7.509766,135,6.51,2502,0,2,no(M2 and SIP pres),2
6,93.01599,1,male,1,4,4,Coma,Coma,1,14,⋯,0.6999512,140,7.659180,135,6.51,2502,0,1,<2 mo. follow-up,1


In [34]:
colonnes_chaine <- sapply(data, is.character)

nombres_vides <- sapply(data[, colonnes_chaine], function(x) sum(x == ""))

# Afficher les résultats
resultats <- data.frame(
  colonne = names(nombres_vides),
  nombre_vides = nombres_vides
)
resultats

,colonne,nombre_vides
,<chr>,<int>
sex,sex,0
dzgroup,dzgroup,0
dzclass,dzclass,0
income,income,2982
race,race,42
ca,ca,0
sfdm2,sfdm2,1400


In [35]:
data <- data[, !names(data) %in% "income"]
#data <- data[, !names(data) %in% "sfdm2"]
data <- data[!(is.na(data$sfdm2) | data$sfdm2 == ""), ]
data <- data[!(is.na(data$race) | data$race == ""), ]
data <- data[!(is.na(data$dnr) | data$dnr == ""), ]

colonnes_chaine <- sapply(data, is.character)

nombres_vides <- sapply(data[, colonnes_chaine], function(x) sum(x == ""))

# Afficher les résultats
resultats <- data.frame(
  colonne = names(nombres_vides),
  nombre_vides = nombres_vides
)
resultats

,colonne,nombre_vides
,<chr>,<int>
sex,sex,0
dzgroup,dzgroup,0
dzclass,dzclass,0
race,race,0
ca,ca,0
sfdm2,sfdm2,0


In [36]:
A <- ifelse(data$hospdead > 0.5, 1, 0)
A <- A[A == 1]
B <- ifelse(data$hospdead < 0.5, 1, 0)
B <- B[B == 1]

A_t = length(A)/nrow(data)
B_t = length(B)/nrow(data)

inv_At = 1/A_t *B_t
inv_Bt = 1

inv_At

[1] 2.325823

In [37]:
library(caret)

set.seed(123)
index <- createDataPartition(data$hospdead, p = 0.7, list = FALSE)

train_data2 <- data[index, ]
test_data2 <- data[-index, ]

In [38]:
weights <- ifelse(train_data2$hospdead == 1, 1, inv_At * 2)

In [39]:
library(glmnet)


In [40]:
formula <- hospdead ~ . - 1

x_train <- model.matrix(formula, data = train_data2)
y_train <- train_data2$hospdead

x_test <- model.matrix(formula, data = test_data2)
y_test <- test_data2$hospdead

In [41]:
ridge_model <- glmnet(x_train, y_train, family = "binomial", alpha = 0)

cv_ridge <- cv.glmnet(x_train, y_train, family = "binomial", alpha = 0)

best_lambda_ridge <- cv_ridge$lambda.min

final_ridge_model <- glmnet(x_train, y_train, family = "binomial", alpha = 0, lambda = best_lambda_ridge, weights = weights)

In [42]:
final_lasso_model <- glmnet(x_train, y_train, family = "binomial", alpha = 1, weights = lasso_weights)

Warning message:
“from glmnet C++ code (error code -79); Convergence for 79th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned”


In [43]:
final_lasso_model


Call:  glmnet(x = x_train, y = y_train, family = "binomial", weights = lasso_weights,      alpha = 1) 

   Df  %Dev   Lambda
1   0  0.00 0.114400
2   1  4.67 0.104200
3   1  8.26 0.094980
4   1 11.08 0.086540
5   3 14.11 0.078850
6   4 18.52 0.071850
7   5 22.65 0.065470
8   5 26.32 0.059650
9   5 29.52 0.054350
10  5 32.35 0.049520
11  5 34.87 0.045120
12  5 37.13 0.041110
13  5 39.18 0.037460
14  6 41.54 0.034130
15  6 43.65 0.031100
16  7 45.54 0.028340
17  9 47.56 0.025820
18  9 49.52 0.023530
19  9 51.24 0.021440
20  9 52.75 0.019530
21  9 54.09 0.017800
22  9 55.28 0.016220
23 10 56.40 0.014780
24 10 57.42 0.013460
25 10 58.32 0.012270
26 10 59.13 0.011180
27 10 59.86 0.010180
28 10 60.51 0.009280
29 11 61.11 0.008455
30 11 61.68 0.007704
31 11 62.19 0.007020
32 11 62.65 0.006396
33 12 63.07 0.005828
34 12 63.45 0.005310
35 14 63.84 0.004838
36 16 64.22 0.004409
37 17 64.59 0.004017
38 18 64.95 0.003660
39 20 65.28 0.003335
40 21 65.59 0.003039
41 21 65.88 0.002769
42 21 66.15 0

In [44]:
#best_lambda_lasso <- final_lasso_model$lambda.min
arg_min <- which.min(final_lasso_model$lambda)
best_lambda_lasso <- final_lasso_model[[5]][arg_min]
#arg_min <- which(final_lasso_model$lambda == best_lambda_lasso)

final_lasso_model <- glmnet(x_train, y_train, family = "binomial", alpha = 1, lambda = best_lambda_lasso, weights = weights)

In [45]:
best_lambda_lasso

[1] 8.857896e-05

In [46]:
print(arg_min)

[1] 78


In [47]:
final_lasso_model


Call:  glmnet(x = x_train, y = y_train, family = "binomial", weights = weights,      alpha = 1, lambda = best_lambda_lasso) 

  Df  %Dev    Lambda
1 50 84.49 8.858e-05

In [48]:
ridge_preds <- predict(final_ridge_model, s = best_lambda_ridge, newx = x_test, type = "response")
ridge_classes <- ifelse(ridge_preds > 0.5, 1, 0)

conf_matrix_ridge <- table(Actual = y_test, Predicted = ridge_classes)

In [49]:
print("Confusion Matrix for Ridge:")
print(conf_matrix_ridge)

accuracy_ridge <- sum(diag(conf_matrix_ridge)) / sum(conf_matrix_ridge)

print(paste("Accuracy for Ridge:", accuracy_ridge))

[1] "Confusion Matrix for Ridge:"
      Predicted
Actual    0    1
     0 1604   14
     1  317  367
[1] "Accuracy for Ridge: 0.856211989574283"


In [ ]:
lasso_preds <- predict(final_lasso_model,s = best_lambda_lasso, newx = x_test, type = "response")
lasso_classes <- ifelse(lasso_preds > 0.5, 1, 0)

conf_matrix_lasso <- table(Actual = y_teresst, Predicted = lasso_classes)

In [51]:
length(y_test)

[1] 2302

In [52]:
length(lasso_classes)

[1] 2302

In [53]:
lasso_preds

,s1
4,1.085244e-15
5,8.677289e-233
11,5.052888e-13
12,3.342403e-01
14,1.001296e-214
15,5.221850e-220
18,9.877574e-01
20,5.889921e-20
21,4.855642e-26
25,7.613980e-46


In [54]:
print("Confusion Matrix for Lasso:")
print(conf_matrix_lasso)

accuracy_lasso <- sum(diag(conf_matrix_lasso)) / sum(conf_matrix_lasso)

print(paste("Accuracy for Lasso:", accuracy_lasso))

[1] "Confusion Matrix for Lasso:"
      Predicted
Actual    0    1
     0 1583   35
     1   77  607
[1] "Accuracy for Lasso: 0.951346655082537"


In [56]:
res <- c()

for (i in 0:10) {
cv_model <- cv.glmnet(
  x_train, y_train,
  alpha = i/10,  # Mélange entre Ridge et Lasso
  family = "binomial",  # Régression logistique
  weights = weights  # Poids pour équilibrer
)

# Extraction du lambda optimal
optimal_lambda <- cv_model$lambda.min
cat("Lambda optimal :", optimal_lambda, "\n")

# Prédictions sur l'ensemble de test
predictions <- predict(cv_model, x_test, s = "lambda.min", type = "response")
predicted_classes <- ifelse(predictions > 0.5, 1, 0)

# Évaluation des performances
confusion_matrix <- table(Predicted = predicted_classes, Actual = y_test)
print(confusion_matrix)

accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
cat(accuracy)
res <- c(res, (accuracy))
}


Lambda optimal : 0.01144041 
         Actual
Predicted    0    1
        0 1596  273
        1   22  411
0.8718506Lambda optimal : 0.0001144041 
         Actual
Predicted    0    1
        0 1583  204
        1   35  480
0.8961772Lambda optimal : 5.720206e-05 
         Actual
Predicted    0    1
        0 1584  190
        1   34  494
0.9026933Lambda optimal : 3.813471e-05 
         Actual
Predicted    0    1
        0 1584  181
        1   34  503
0.906603Lambda optimal : 2.860103e-05 
         Actual
Predicted    0    1
        0 1585  170
        1   33  514
0.9118158Lambda optimal : 2.288082e-05 
         Actual
Predicted    0    1
        0 1583  156
        1   35  528
0.9170287Lambda optimal : 1.906735e-05 
         Actual
Predicted    0    1
        0 1583  150
        1   35  534
0.9196351Lambda optimal : 1.634345e-05 
         Actual
Predicted    0    1
        0 1584  123
        1   34  561
0.9317984Lambda optimal : 1.722502e-05 
         Actual
Predicted    0    1
        

Warning message:
“from glmnet C++ code (error code -79); Convergence for 79th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned”
Warning message:
“from glmnet C++ code (error code -79); Convergence for 79th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned”
Warning message:
“from glmnet C++ code (error code -79); Convergence for 79th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned”
Warning message:
“from glmnet C++ code (error code -79); Convergence for 79th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned”
Warning message:
“from glmnet C++ code (error code -78); Convergence for 78th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned”
Warning message:
“from glmnet C++ code (error code -79); Convergence for 79th lambda value not reached after maxit=100000 iterations; s

Lambda optimal : 8.857896e-05 
         Actual
Predicted    0    1
        0 1582   77
        1   36  607
0.9509123

alpha = 0.1 : acc = 0.91
alpha = 0.2 : acc = 0.92
alpha = 0.3 : acc = 0.93
alpha = 0.4 : acc = 0.93
alpha = 0.5 : acc = 0.93
alpha = 0.6 : acc = 0.94
alpha = 0.7 : acc = 0.94
alpha = 0.8 : acc = 0.95
alpha = 0.9 : acc = 0.95